In [4]:
from __future__ import division, print_function
import numpy as np
import os
import sys
sys.path.append('../hw5/homework5_code')
from hmmlearn.hmm import MultinomialHMM

In [281]:
%run 'preprocessing.ipynb'

In [282]:
def get_rhymes(words):
    num_words = len(words)
    rhymes = np.empty((num_words, num_words))
    
    for i in range(num_words):
        for j in (k for k in range(num_words) if not k == i):
            if poetrytools.rhymes(words[i], words[j]):
                rhymes[i][j] = 1
    return rhymes

In [283]:
quatrain_rhymes = get_rhymes(quatrain_words)
volta_rhymes = get_rhymes(volta_words)
couplet_rhymes = get_rhymes(couplet_words)

In [284]:
reversed_quatrain_model = MultinomialHMM(n_components=20)
reversed_volta_model = MultinomialHMM(n_components=20)
reversed_couplet_model = MultinomialHMM(n_components=20)
reversed_quatrain_model.fit(np.concatenate(reversed_converted_quatrain_lines), quatrain_lengths)
reversed_volta_model.fit(np.concatenate(reversed_converted_volta_lines), volta_lengths)
reversed_couplet_model.fit(np.concatenate(reversed_converted_couplet_lines), couplet_lengths)

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=20,
        n_iter=10, params='ste',
        random_state=<mtrand.RandomState object at 0x10dabd640>,
        startprob_prior=1.0, tol=0.01, transmat_prior=1.0, verbose=False)

In [364]:
def generate_b_line(seed, rhymes, model, length=7):
    b_line = []
    start_probs = model.startprob_
    emission_probs = model.emissionprob_
    transition_probs = model.transmat_

    start_state = np.random.choice(len(start_probs), p = start_probs)
    
    possible_start_emissions = np.where(rhymes[seed] == 1)
    probs = np.array(emission_probs[start_state][possible_start_emissions])

    scaled_probs = probs / sum(probs)
    
    start_emission = np.random.choice(possible_start_emissions[0], p=scaled_probs)
    b_line.append(start_emission)
    
    curr_state = start_state
    for _ in range(length - 1):
        possible_transitions = transition_probs[curr_state]
        curr_state = np.random.choice(len(possible_transitions), p=possible_transitions)
        possible_emissions = emission_probs[curr_state]
        curr_emission = np.random.choice(len(possible_emissions), p=possible_emissions)
        b_line.append(curr_emission)
        
    return b_line

def convert_a_line(sample, words):
    ret = ''
    reversed_list = []
    for word in np.nditer(sample):
        reversed_list.append(word)
    
    i = 0
    for word in reversed(reversed_list):
        curr_word = words[word]
        if i == 0 or len(curr_word) == 1:
            ret += curr_word.title() + ' '
        else:
            ret += curr_word + ' '
        i += 1
    return ret

def convert_b_line(sample, words):
    ret = ''
    i = 0
    for word in reversed(sample):
        curr_word = words[word]
        if i == 0 or len(curr_word) == 1:
            ret += curr_word.title() + ' '
        else:
            ret += curr_word + ' '
        i += 1
    return ret

def generate_pair(model, rhymes, length=7):
    while True:
        a_line = model.sample(length)[0]
        seed = a_line[0][0]
        if len(np.where(rhymes[seed] == 1)[0]) > 0:
            b_line = generate_b_line(seed, rhymes, model)
            return a_line, b_line
        
def generate_rhyming_sonnet():
    sonnet = ''
    a_lines = []
    b_lines = []
    
    for _ in range(4):
        a_line, b_line = generate_pair(reversed_quatrain_model, quatrain_rhymes)
        a_lines.append(a_line)
        b_lines.append(b_line)
    
    for i in range(2):
        sonnet += convert_a_line(a_lines[2 * i], quatrain_words) + '\n'
        sonnet += convert_a_line(a_lines[2 * i + 1], quatrain_words) + '\n'
        sonnet += convert_b_line(b_lines[2 * i], quatrain_words) + '\n'
        sonnet += convert_b_line(b_lines[2 * i + 1], quatrain_words) + '\n'
    
    a_lines = []
    b_lines = []
    
    for _ in range(2):
        a_line, b_line = generate_pair(reversed_volta_model, volta_rhymes)
        a_lines.append(a_line)
        b_lines.append(b_line)
    
    sonnet += convert_a_line(a_lines[0], volta_words) + '\n'
    sonnet += convert_a_line(a_lines[1], volta_words) + '\n'
    sonnet += convert_b_line(b_lines[0], volta_words) + '\n'
    sonnet += convert_b_line(b_lines[1], volta_words) + '\n'
    
    a_line, b_line = generate_pair(reversed_couplet_model, couplet_rhymes)
    sonnet += convert_a_line(a_line, couplet_words) + '\n'
    sonnet += convert_b_line(b_line, couplet_words) + '\n'
    
    return sonnet

def generate_10_rhyming_sonnets():
    sonnets = ''
    for i in range(10):
        sonnets += str(i) + '\n' + generate_rhyming_sonnet() + '\n'
    
    f = open("project2data/rhyming_shakespeare.txt","w")
    f.write(sonnets)
    return sonnets

In [365]:
print(generate_10_rhyming_sonnets())

0
To why curse but art your you 
When of and powerful and doing cruel 
Hath in decays thou all eyes hue 
Mine my glorious but of when will 
Me than ornament hath your extant place 
Bounteous my my all sake ah alone 
That my these laid for my chase 
Wherein of and than me not own 
May several look of is what rhyme 
Who an false age's sin so veins 
To thy and trial such with crime 
For before and be to familiar gains 
He cupid put that we art spent 
Love love shall those in whose content 

1
Willing fair in then the as never 
Shall men again as from those been 
Broils pure and trifles on I lover 
None every chronicle of thee how forgotten 
Hath self still A death wherefore discontent 
Blamed winter's such or what wane deem 
Sets the am the me self invent 
Phrase you have that that they esteem 
Do not I she being compounded bright 
Birth on in with making keeps part 
I clouds how is herald twofold spite 
And look say even glowing in art 
Have speaking thou but thou my heart 
Him take he t